In [27]:
import pandas as pd
import altair as alt
from vega_datasets import data

In [6]:
path = './data'
airlineTimeFile = 'airline_time.csv'
routeFile = 'routes.csv'
allFile = 'data.csv'
#allDf = pd.read_csv('%s/%s' % (path, allFile))
routeDf = pd.read_csv('%s/%s' % (path, routeFile))
airlineTimeDf = pd.read_csv('%s/%s' % (path, airlineTimeFile))

In [11]:
#airlineTimeDf['OP_UNIQUE_CARRIER'].unique()
airlines = ['WN','AA','OO','DL','UA','B6','YX','NK']
df_time = airlineTimeDf.copy()
df_time = df_time[df_time["ARR_DELAY"] < 400]
df_time = df_time[df_time["OP_UNIQUE_CARRIER"].isin(airlines)]
df_time = df_time.sample(5000, random_state=0)
df_time['ARR_DELAY'].max()

395.0

### Plot first figure

In [24]:
pts = alt.selection(type="single", encodings=['x'])

rect = alt.Chart(df_time).mark_rect().encode(
            alt.Y("ARR_DELAY:Q", bin=True),
            alt.X("QUARTER:N"),
            color="count()"
        ).transform_filter(
            pts
        )
rect.width = 750
rect.height = 400

circle = alt.Chart(df_time).transform_bin(
            "ARR_DELAY_bin", field="ARR_DELAY"
        ).transform_filter(
            pts
        ).transform_joinaggregate(
            total="count()",
            groupby=["ARR_DELAY_bin"]
        ).transform_joinaggregate(
            in_group="count()",
            groupby=["ARR_DELAY_bin", "QUARTER"]
        ).transform_calculate(
            PERCENT_BY_ARR_DELAY=alt.datum.in_group / alt.datum.total
        ).mark_circle().encode(
            alt.Y("ARR_DELAY:Q", bin=True, axis=alt.Axis(title="Delay Time (minutes)", titleFontSize=14)),
            alt.X("QUARTER:N", axis=alt.Axis(title="Quarter", titleFontSize=14, labelAngle=0)),
            alt.Size("PERCENT_BY_ARR_DELAY:Q", scale=alt.Scale(range=[0, 2000]), legend=alt.Legend(format='%', title='Percentage')),
            tooltip=['QUARTER:N', 'count()', alt.Tooltip('PERCENT_BY_ARR_DELAY:Q', format='.2f'),]
        )
circle.width = 750
circle.height = 400

bar = alt.Chart(df_time).mark_bar().encode(
    x=alt.X('OP_UNIQUE_CARRIER:N', sort='-y', axis=alt.Axis(title="Operating Carrier", labelAngle=0, titleFontSize=14)),
    y=alt.Y('count()', axis=alt.Axis(titleFontSize=14)),
    color=alt.condition(pts, alt.ColorValue("steelblue"), alt.ColorValue("grey")),
    tooltip=['OP_UNIQUE_CARRIER:N', 'count()']
).properties(
    width=750,
    height=200
).add_selection(pts)

alt.vconcat(
    rect + circle,
    bar
).resolve_legend(
    color="independent",
    size="independent"
)

alt.VConcatChart(...)

### Route

In [ ]:
routeDf.head()

,ORIGIN,ORIGIN_STATE,ORIGIN_LAT,ORIGIN_LONG,DEST,DEST_STATE,DEST_LAT,DEST_LONG,ARR_DELAY
0,BOS,MA,42.36435,-71.00518,TPA,FL,27.97547,-82.53325,20.0
1,SDF,KY,38.17439,-85.73600,ATL,GA,33.64044,-84.42694,267.0
2,SRQ,FL,27.39533,-82.55411,MSP,MN,44.88055,-93.21692,46.0
3,MSP,MN,44.88055,-93.21692,BWI,MD,39.17540,-76.66820,19.0
4,ORD,IL,41.97960,-87.90446,ATL,GA,33.64044,-84.42694,32.0


In [74]:
df_routes = routeDf.copy() 
df_routes = df_routes[df_routes["ARR_DELAY"] < 400]
df_routes = df_routes.sample(5000, random_state=0)

In [57]:
df_routes[(df_routes['ORIGIN']=='DEN') & (df_routes['DEST']=='LAX')]

,ORIGIN,ORIGIN_STATE,ORIGIN_LAT,ORIGIN_LONG,DEST,DEST_STATE,DEST_LAT,DEST_LONG,ARR_DELAY
195151,DEN,CO,39.85841,-104.667,LAX,CA,33.94254,-118.40807,50.0
604960,DEN,CO,39.85841,-104.667,LAX,CA,33.94254,-118.40807,42.0
302620,DEN,CO,39.85841,-104.667,LAX,CA,33.94254,-118.40807,15.0
538088,DEN,CO,39.85841,-104.667,LAX,CA,33.94254,-118.40807,24.0
72722,DEN,CO,39.85841,-104.667,LAX,CA,33.94254,-118.40807,22.0


In [77]:
states = alt.topo_feature(data.us_10m.url, feature="states")

background = alt.Chart(states).mark_geoshape(
    fill="lightgray",
    stroke="white"
).properties(
    width=750,
    height=500
).project("albersUsa")

flights_airport = df_routes
select_city = alt.selection_single(
    on="mouseover", nearest=True, fields=["ORIGIN"], empty="none"
)

connections = alt.Chart(flights_airport
).transform_filter(
    (alt.datum.ORIGIN_STATE != "PR") & (alt.datum.ORIGIN_STATE != "VI") & (alt.datum.DEST_STATE != "PR") & (alt.datum.DEST_STATE != "VI")
).transform_joinaggregate(
    Count="count()",
    Avg_Delay='mean(ARR_DELAY)',
    groupby=["ORIGIN", "DEST"]
).mark_rule(opacity=0.35).encode(
    latitude="ORIGIN_LAT:Q",
    longitude="ORIGIN_LONG:Q",
    latitude2="DEST_LAT:Q",
    longitude2="DEST_LONG:Q",
    size=alt.Size("Count:Q", scale=alt.Scale(range=[0, 500]), legend=None),
    color=alt.Color("Avg_Delay:Q", scale=alt.Scale(scheme='yelloworangebrown', domain=[0, 200]), legend=alt.Legend(title='Average Delay (min)'))
).transform_filter(
    select_city
)

points = alt.Chart(flights_airport
).transform_filter(
    (alt.datum.ORIGIN_STATE != "PR") & (alt.datum.ORIGIN_STATE != "VI") & (alt.datum.DEST_STATE != "PR") & (alt.datum.DEST_STATE != "VI")
).transform_joinaggregate(
    Total_Flights="count()",
    groupby=["ORIGIN"]
).mark_circle().encode(
    latitude="ORIGIN_LAT:Q",
    longitude="ORIGIN_LONG:Q",
    size=alt.Size("Total_Flights:Q", scale=alt.Scale(range=[0, 1000]), legend=None),
    order=alt.Order("Total_Flights:Q", sort="descending"),
    tooltip=["ORIGIN:N", "Total_Flights:Q"]
).add_selection(
    select_city
)

(background + connections + points).configure_view(stroke=None)

alt.LayerChart(...)